In [1]:
import tensorflow as tf
from tensorflow import keras
from loguru import logger
from tensorflow.keras.models import Model
import cv2
import numpy as np

2022-11-10 19:39:41.942399: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
train_den = "/home/denmann99/autoencoder/pics/bryan1"
train_uch = "/home/denmann99/autoencoder/pics/uchtdorf1"
denver=tf.keras.preprocessing.image_dataset_from_directory(train_den,shuffle=True,
                                                     batch_size=64,image_size=(256, 256),seed=123)
uch=tf.keras.preprocessing.image_dataset_from_directory(train_uch,shuffle=True,
                                                     batch_size=64,image_size=(256, 256),seed=123)
def normalizer(generator):
  normalization_layer = tf.keras.layers.experimental.preprocessing.Rescaling(1./255)

  def change_inputs(images, labels):
    x = tf.image.resize(normalization_layer(images),[256, 256], method=tf.image.ResizeMethod.NEAREST_NEIGHBOR)
    return x, x

  return generator.map(change_inputs)
normalized_denver = normalizer(denver)
normalized_uchtdorf = normalizer(uch)

Found 4697 files belonging to 1 classes.
Found 1823 files belonging to 1 classes.


2022-11-10 19:39:42.888199: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-11-10 19:39:42.888744: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-11-10 19:39:42.924695: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-10 19:39:42.924924: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2060 computeCapability: 7.5
coreClock: 1.56GHz coreCount: 30 deviceMemorySize: 5.79GiB deviceMemoryBandwidth: 245.91GiB/s
2022-11-10 19:39:42.924939: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-11-10 19:39:42.926314: I tensorflow/stream_executor/platform/def

In [3]:
encoder_input = tf.keras.layers.Input(shape=(256, 256, 3))

# Encoder
x = tf.keras.layers.Conv2D(32, (3, 3), padding="same")(encoder_input)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(64, (3, 3), padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(128, (3, 3), padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(256, (3, 3), padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
encoder_output = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
encoder= keras.Model(inputs=[encoder_input], outputs=[encoder_output])



# Decoder
decoder_input = tf.keras.layers.Input(shape=(16,16,256))
x = tf.keras.layers.Conv2DTranspose(256, (3, 3), strides=2, padding="same")(decoder_input)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=2, padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=2, padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
decoder_output = tf.keras.layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)
decoder= keras.Model(inputs=[decoder_input], outputs=[decoder_output])


# Autoencoder
input_layer = tf.keras.layers.Input(shape=(256,256,3))
encoder_out = encoder.call(input_layer)
decoder_out = decoder.call([encoder_out])
ae_model = keras.Model(inputs=[input_layer], outputs=[decoder_out])
ae_model.compile(optimizer=keras.optimizers.Adam(learning_rate=.001), loss='mse')
print(ae_model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 64)      18496     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 64)        0   

In [7]:
ae_model.fit(
    x=normalized_denver,
    epochs=10,
    batch_size=1,
    shuffle=True
)
ae_model.save("bryans_ae.h5")

Epoch 1/10


2022-11-10 19:42:04.459188: W tensorflow/core/common_runtime/bfc_allocator.cc:433] Allocator (GPU_0_bfc) ran out of memory trying to allocate 512.00MiB (rounded to 536870912)requested by op gradient_tape/model_2/conv2d_4/Conv2D/Conv2DBackpropInput
Current allocation summary follows.
2022-11-10 19:42:04.459271: I tensorflow/core/common_runtime/bfc_allocator.cc:972] BFCAllocator dump for GPU_0_bfc
2022-11-10 19:42:04.459303: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (256): 	Total Chunks: 53, Chunks in use: 53. 13.2KiB allocated for chunks. 13.2KiB in use in bin. 2.5KiB client-requested in use in bin.
2022-11-10 19:42:04.459323: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (512): 	Total Chunks: 6, Chunks in use: 6. 3.0KiB allocated for chunks. 3.0KiB in use in bin. 3.0KiB client-requested in use in bin.
2022-11-10 19:42:04.459341: I tensorflow/core/common_runtime/bfc_allocator.cc:979] Bin (1024): 	Total Chunks: 7, Chunks in use: 7. 7.2KiB allocated for chunk

ResourceExhaustedError:  OOM when allocating tensor with shape[64,32,256,256] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/model_2/conv2d_4/Conv2D/Conv2DBackpropInput (defined at tmp/ipykernel_38606/887432999.py:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1524]

Function call stack:
train_function


In [34]:
# face = cv2.imread("/home/denmann99/autoencoder/pics/uchtdorf1/uchtdorf/uchtdorf_face2.jpg")
# face = cv2.resize(face,(256,256))
# face = np.reshape(face, (1,256,256,3))
# face = face/255
# pred = ae_model.predict(face)
# pred = np.reshape(pred,(256,256,3))
# pred = pred*255
# cv2.imwrite("/home/denmann99/autoencoder/Capturing.png", pred)

In [35]:
encoder_input_uch = tf.keras.layers.Input(shape=(256, 256, 3))

# Encoder
x = tf.keras.layers.Conv2D(32, (3, 3), padding="same")(encoder_input_uch)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(64, (3, 3), padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(128, (3, 3), padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
x = tf.keras.layers.Conv2D(256, (3, 3), padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
encoder_output_uch = tf.keras.layers.MaxPooling2D((2, 2), padding="same")(x)
encoder_uch= keras.Model(inputs=[encoder_input_uch], outputs=[encoder_output_uch])



# Decoder
decoder_input_uch = tf.keras.layers.Input(shape=(16,16,256))
x = tf.keras.layers.Conv2DTranspose(256, (3, 3), strides=2, padding="same")(decoder_input_uch)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=2, padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=2, padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
x = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=2, padding="same")(x)
x = tf.keras.layers.LeakyReLU(alpha = 0.1)(x)
decoder_output_uch = tf.keras.layers.Conv2D(3, (3, 3), activation="sigmoid", padding="same")(x)
decoder_uch= keras.Model(inputs=[decoder_input_uch], outputs=[decoder_output_uch])


# Autoencoder
input_layer_uch = tf.keras.layers.Input(shape=(256,256,3))
encoder_out_uch = encoder.call(input_layer_uch)
decoder_out_uch = decoder.call([encoder_out_uch])
ae_model_uch = keras.Model(inputs=[input_layer_uch], outputs=[decoder_out_uch])
ae_model_uch.compile(optimizer=keras.optimizers.Adam(learning_rate=.001), loss='mse')
print(ae_model_uch.summary())

Model: "model_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_35 (InputLayer)        [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 256, 256, 32)      896       
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 128, 128, 32)      9248      
_________________________________________________________________
leaky_re_lu_17 (LeakyReLU)   (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_26 (MaxPooling (None, 64, 64, 32)        0  

In [36]:
ae_model_uch.fit(
    x=normalized_uchtdorf,
    epochs=10,
    batch_size=64,
    shuffle=True
)
ae_model_uch.save("uchtdorf_ae.h5")

Epoch 1/10
29/29 [==============================] - 15s 470ms/step - loss: 0.0087
Epoch 2/10
29/29 [==============================] - 14s 462ms/step - loss: 0.0036
Epoch 3/10
29/29 [==============================] - 14s 459ms/step - loss: 0.0029
Epoch 4/10
29/29 [==============================] - 14s 462ms/step - loss: 0.0027
Epoch 5/10
29/29 [==============================] - 15s 476ms/step - loss: 0.0026
Epoch 6/10
29/29 [==============================] - 14s 466ms/step - loss: 0.0025
Epoch 7/10
29/29 [==============================] - 14s 460ms/step - loss: 0.0025
Epoch 8/10
29/29 [==============================] - 15s 486ms/step - loss: 0.0024
Epoch 9/10
29/29 [==============================] - 16s 518ms/step - loss: 0.0023
Epoch 10/10
29/29 [==============================] - 15s 483ms/step - loss: 0.0023


In [37]:
face = cv2.imread("/home/denmann99/autoencoder/pics/uchtdorf1/uchtdorf/uchtdorf_face2.jpg")
face = cv2.resize(face,(256,256))
face = np.reshape(face, (1,256,256,3))
face = face/255
uch_pred = encoder_uch.predict(face)
output = decoder.predict(uch_pred)
output = np.reshape(output,(256,256,3))
output = output*255
cv2.imwrite("/home/denmann99/autoencoder/uch2bryan.png", output)

2022-11-10 19:32:00.628022: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED


2022-11-10 19:32:01.251407: E tensorflow/stream_executor/cuda/cuda_blas.cc:226] failed to create cublas handle: CUBLAS_STATUS_NOT_INITIALIZED


True